In [1]:
## Sometimes Jupyter notebook doesn't retain your PATH environment variable -- this will mess up a number of things.
## We recommend specifying the environment variable manually here
import os
os.environ["PATH"] = '/home/mattjones/bin:/home/mattjones/.local/bin:/home/mattjones/myapps/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/mattjones/opt/gurobi801/linux64/bin:/home/mattjones/software/bowtie2-2.3.4.2:/home/mattjones/emboss/EMBOSS-6.6.0/emboss'

from cassiopeia.TreeSolver import utilities

import pandas as pd 
import numpy as np
import networkx as nx
import pickle as pic

import sys

## Introduction

The minimum input for reconstructing trees with Cassiopeia is a **character matrix** , consisting of indel measurements for **M** *characters* across **N** cells. Here, the term "character" is an abstract term referring to variables containing information about where cells lie in some phylogeny. In our technology, these are individual target sites -- i.e. those sites targeted by Cas9 and subsequently accumulate indels. More generally, these can be nulceotide positions in DNA or even phenotypic observations of an animal. 

We use this abstract data structure because it is general enough for any technology, and does not rely on any assumption other than one can *phase* the observations (i.e. you have the ability to index mutations by where they occurred). 

Here, we begin by describing how one takes an allele table from our PreProcessing Pipeline and converting into a character matrix. To do so, we not only specify an allele table but also a target lineage group (i.e. clone) to reconstruct

In [2]:
alleletable_fp = "lg_output/test.alleleTable.txt" # relative file path to the allele table
alleletable = pd.read_csv(alleletable_fp, sep='\t')

# here we're assuming there exists a 'lineage group' that's annotated in the allele table as LG4. 
# You can imagine subsetting your allele table differentely 
target_lg = 4 

### Estimate Allele Proportions (i.e. State Priors)

As a brief detour, we can estimate the indel priors from the allele tables by measuring the proportion of intBCs that report a given indel across independent clones. The idea behind this is that we'd like to subset the data into groups of independent sets (i.e. those that are not related to one another at all) as this will eliminate the possiblity of "double counting" indels because they were inherited. Inevitably, we will underestimate the propensity of a given indel to occur if it happened many times in the same location (as we choose to assume this only occurs by inheritance); however, the more independent groups we have, the greater our ability to estimate priors.

The function ``get_indel_props`` will do this and takes the following arguments:

- **at**: Allele Table Dataframe
- **group_var**: A list of variables (columns in the allele table) by which to subset the data into independent groups (Default = ['intBC']). If you have redundant intBCs, you can also partition your data on the basis of intBCs and clone annotations (e.g. group_var = ['intBC', 'clone'])

This will return a dataframe with three columns -- the indel identity, the proportion of groups it appeared in, and the frequency of this indel (an estimate of the prior probability).

In [3]:
allele_props = utilities.get_indel_props(alleletable)

Counting unique alleles: 100%|██████████| 124/124 [00:00<00:00, 9964.44it/s]


### Create character matrix from a lineage group

We now aggregate our observations from the allele table into a character matrix -- a matrix whose dimensions are `n x m`, where we have `n` cells and `m` characters (in our case, this is the number of total target sites, 3 times the number of integrations). We can use the ``alleletable_to_character_matrix`` function here, which takes the following parameters:

- **at**: alleletable (subsetted for a particular lineage group) 
- **out_fp**: the output file path for the resulting character matrix (Default = None). This value is only necessary when ``write = True``.
- **mutation_map**: the indel priors, calculated as above with ``get_indel_props``. If nothing is provided, no prior probabilities will be used for later tree reconstructions.
- **no_context**: Use indels without context (default = False). If True, this means that the character matrix will only look at indel identity, not the context in which it appears.
- **write**: Write to file (default = True).
- **to_drop**: A set of intBCs that you'd like to ignore while creating a character matrix. 
- **allele_rep_thresh**: A maximum threshold of allele-dominance in a given character (default = 1.0). If one allele (i.e. indel) appears at higher than this proportion in a given character, the character is deemed uninformative and thrown out.  

This function will return (or write) three files: 
- the character matrix
- the indel proprotions dictionary, specifying the mutation probabilities for each character to a particular state
- dictionary translating a state-character pair to an observed allele. 

If write is True, the last two files are saved as pickle files, and only written if a mutation_map is provided. Else, the dictionaries are returned. 

In [4]:
lg = alleletable[alleletable["lineageGrp"] == target_lg]

utilities.alleletable_to_character_matrix(lg, "test_lg4_character_matrix.txt", mutation_map=allele_props, write=True)

Processing characters: 100%|██████████| 45/45 [00:00<00:00, 1300.82it/s]


### Reconstruct a lineage

You can now reconstruct lineages using the `reconstruct-lineage` command, which takes in many different options depending on which of many lineage solvers you'd like to use. We provide Neighbor-Joining, Camin-Sokal (implemented with PHYLIP), greedy, Steiner-Tree/ILP, and Hybrid algorithms. Use the `-h` flag to see all possible parameters, or take a look at this walkthrough for tree reconstruction: [link to tree documentation].

The output of this procedure will be a Cassiopeia_Tree object, which is essentially a wrapper for a networkx object representing the Tree, and a newick string that can be read in using a variety of different software. This object also contains several other utilities functions which can be found on our documentation website: https://cassiopeia-lineage.readthedocs.io/en/latest/Cassiopeia.TreeSolver.html

### Post-Processing a Tree

Here we'll work with a tree named `test_lg4_tree.pkl`, and we assume it was reconstructed using the `reconstruct-lineage` command line tool and associated functions. If this is the case, Cassiopeia has returned what we call a *state-tree*: because multiple cells can report the same set of alleles, or states, Cassiopeia reconstructs over the unique states rather than all cells. It thus returns a tree that proposes a phylogenetic relationship over these states. Taken together, we'll need to append cells back to the leaves of the tree using the `post-process` functionality. This does three things:

1. Map terminal character states to the cell identifiers
2. Add "Redundant" leaves to the terminal leaves. This is necessary because if not ever cell represents a unique state, then the final tree would only be over a subset of the cells originally in the character matrix. This is known as the Post-Processing Tree Step.
3. If there exist any "leaves" that do not correspond to character states observed in the character matrix, these are pruned back. 

While we provide the python code to do this, you can also take advantage of the `post-process-tree`. Use the `-h` flag to see options and usage.

#### Mapping Terminal states

In [26]:
g = nx.read_gpickle("test_lg4_tree.pkl") # read in a Cassiopeia-Tree
cm = pd.read_csv("test_lg4_character_matrix.txt", sep='\t', index_col = 0) # read in the associated character matrix

#### Post-Process Tree & Add Redundant Leaves

In [27]:
g = g.post_proces(cm = cm) # Cassiopeia-Tree's have built in functionality to post-process, callable as a method on the object 

# create a newick string for the given post-processed tree. Note that you need to pass in the 
# networkx object for this

g.newick = utilities.convert_network_to_newick_format(g.network) 

removing character strings from sample names: 100%|██████████| 1048/1048 [00:00<00:00, 581509.54it/s]


In [28]:
## now save final tree
out_fp = "test_lg4_tree.processed.pkl"

pic.dump(g, open(out_fp, 'wb'))

## Final notes

Congrats! You've successfully created a character matrix, reconstructed a tree, and post-processed it. You are now at the stage where you can make hypotheses regarding how cells are related to one another, and if you have paired phenotypic measurements for each cell (e.g. single-cell RNAseq), you can start integrating these two orthogonal measurements to see how phenotypes are inherited. 

To get you started, we have some key suggestions - 

1. We suggest you take a look at the concordance between alleles and trees. This can be done two ways:
    - This can be done by plotting an "indel heatmap", which we've provided a script for creating in scripts/alleletable_2_heatmap.py. This will take in an allele table and produce a data structure that resembles a character matrix though it retains the indel identities. You can plot this indel heatmap using [ITOL](https://itol.embl.de/), or another package like [phytools](http://blog.phytools.org/). 
    - You can explore the relationship between "allele distance" and "phylogenetic distance". We have provided functionality for computing both of these items simultaneously with `utiliites.compute_pairwise_dist_nx`.
    
2. As noted above, there are several ways to visualize the tree. We've found success, especially with larger trees, with [ITOL](https://itol.embl.de/).

3. There are several packages for doing tree-based analysis that you can use to study evolutionary relationships between cells. We've found the following usefule: [ETE3](http://etetoolkit.org/), [Biopython](https://biopython.org/wiki/Phylo), and [scikit-bio](http://scikit-bio.org/). These have nice functionality around traversing a tree, visualizing sub-clades, and more. All of them have functionality for reading in trees from newick format. 

4. On integrating RNA-seq and tree information, we recommend using [Hotspot](https://yoseflab.github.io/Hotspot/) which has a module specifically for phylogenetic trees. 

5. We are currently in the process of building ou the cassiopeia.Analysis module, which thus far only has functionality for running *FitchCount* which quantifies how plastic different states are in a tree given the states at the leaves. You can read more about this in our recent [manuscript](https://www.biorxiv.org/content/10.1101/2020.04.16.045245v2). 